In [ ]:
# Installation des dépendances nécessaires pour OpenAIServerModel
!pip install 'smolagents[openai]'

In [ ]:
from smolagents import CodeAgent, ToolCallingAgent, DuckDuckGoSearchTool, VisitWebpageTool, OpenAIServerModel
import os

model = OpenAIServerModel(
    model_id="gpt-4o",
    api_base="https://api.openai.com/v1",
    api_key=os.environ["OPENAI_API_KEY"],
)

# Définir l'agent web avec un nom et une description
web_agent = ToolCallingAgent(
    tools=[DuckDuckGoSearchTool(), VisitWebpageTool()],
    model=model,
    name="search_agent",
    description="Performs web searches using DuckDuckGo and visits webpages."
)

# Donner un nom unique à l'agent manager pour éviter le doublon
manager_agent = CodeAgent(
    tools=[], 
    model=model, 
    name="manager_agent",
    managed_agents=[web_agent]
)

manager_agent.run("Who is the CEO of Hugging Face?")

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, TransformersEngine

model_name = "HuggingFaceTB/SmolLM2-1.7B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

engine = TransformersEngine(pipe)
engine([{"role": "user", "content": "How are you ?"}], stop_sequences=["great"])


In [ ]:
# Test de l'agent CSV sans le paramètre problématique
from smolagents import ToolCallingAgent, OpenAIServerModel
from managed_agent.csv_analyzer_tool import CSVAnalyzerTool
import os

# Création d'un exemple de contenu CSV simple
csv_content = """nom,age,ville
Alice,25,Paris
Bob,30,Lyon
Claire,28,Marseille
David,35,Bordeaux"""

# Création du modèle
model = OpenAIServerModel(
    model_id="gpt-4o",
    api_base="https://api.openai.com/v1",
    api_key=os.environ["OPENAI_API_KEY"],
)

# Création d'un agent data analyst avec l'outil d'analyse CSV
# En supprimant le paramètre problématique
data_agent = ToolCallingAgent(
    tools=[CSVAnalyzerTool()],
    model=model,
    name="data_analyst",
    description="Analyzes CSV data."
)

# Création d'un agent manageur
manager = CodeAgent(
    tools=[],
    model=model, 
    name="manager_agent",
    managed_agents=[data_agent]
)

# Test de l'agent
result = manager.run(f"Voici un fichier CSV à analyser:\n{csv_content}\n\nPouvez-vous me fournir des informations sur ce fichier?")
print(result)